In [66]:
import pandas as pd
from functools import reduce
import numpy as np
from pandas import merge

In [67]:

icu_cases_path =  'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/icu.csv'
vax_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/vaccination/vax_malaysia.csv'
death_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/deaths_malaysia.csv'
cases_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv'

In [68]:
beginningDate = '2021-01-01'
endingDate = '2021-06-30'

In [69]:
icu_cases = pd.read_csv(icu_cases_path, usecols= ['date', 'state', 'beds_icu_covid'], parse_dates= ['date'])
vax_malaysia = pd.read_csv(vax_malaysia_path)
death_malaysia = pd.read_csv(death_malaysia_path)
cases_malaysia = pd.read_csv(cases_malaysia_path)

icu_cases['date'] = pd.to_datetime(icu_cases['date'])
vax_malaysia['date'] = pd.to_datetime(vax_malaysia['date'])

icu_cases = icu_cases[icu_cases['date'] >= beginningDate]
icu_cases = icu_cases[icu_cases['date'] <= endingDate]

vax_malaysia = vax_malaysia[vax_malaysia['date'] >= beginningDate]
vax_malaysia = vax_malaysia[vax_malaysia['date'] <= endingDate]

death_malaysia = death_malaysia[death_malaysia['date'] >= beginningDate]
death_malaysia = death_malaysia[death_malaysia['date'] <= endingDate]

cases_malaysia = cases_malaysia[cases_malaysia['date'] >= beginningDate]
cases_malaysia = cases_malaysia[cases_malaysia['date'] <= endingDate]
#print(vax_malaysia.head(5))

In [95]:
casesAgesChild = cases_malaysia.iloc[:, 10]
casesAgesAdole = cases_malaysia.iloc[:, 11]
casesAgesAdult = cases_malaysia.iloc[:, 12]
casesAgesElder = cases_malaysia.iloc[:, 13]
#casesNum = cases_malaysia.iloc[:, 1]
casesUnvax = cases_malaysia.iloc[:, 6]
casesPvax = cases_malaysia.iloc[:, 7]
casesFvax = cases_malaysia.iloc[:, 8]




vaxStatus = pd.DataFrame(cases_malaysia, columns= ['date', 'cases_unvax', 'cases_pvax', 'cases_fvax'])
icuCasesTotal = pd.DataFrame(icu_cases, columns= ['date', 'beds_icu_covid'])
allAgesCases = pd.DataFrame(cases_malaysia, columns= ['date', 'cases_elderly', 'cases_adult', 'cases_adolescent', 'cases_child'])
casesNum = pd.DataFrame(cases_malaysia, columns= ['date', 'cases_new'])

vaxStatus = vaxStatus.groupby('date').sum()
icuCasesTotal = icuCasesTotal.groupby('date').sum()
allAgesCases = allAgesCases.groupby('date').sum()



deathNum = death_malaysia.iloc[:, 1]
deathUnvax = death_malaysia.iloc[:, 5]
deathPvax = death_malaysia.iloc[:, 6]
deathFvax = death_malaysia.iloc[:, 7]




features = casesNum.merge(allAgesCases, on='date', how=  'left')
features = features.merge(vaxStatus, on= 'date', how= 'left')



print(features)



           date  cases_new  cases_elderly  cases_adult  cases_adolescent  \
0    2021-01-01       2068            115         1715                78   
1    2021-01-02       2295            113         1895                60   
2    2021-01-03       1704            114         1307                71   
3    2021-01-04       1741            103         1346                72   
4    2021-01-05       2027            125         1610                87   
..          ...        ...            ...          ...               ...   
176  2021-06-26       5803            464         4280               326   
177  2021-06-27       5586            424         4174               316   
178  2021-06-28       5218            385         3755               348   
179  2021-06-29       6437            517         4191               367   
180  2021-06-30       6276            507         4401               377   

     cases_child  cases_unvax  cases_pvax  cases_fvax  
0            129         2068  

In [ ]:
cases_malaysia = pd.read_csv(cases_malaysia_path)
sorted_cases_malaysia = pd.DataFrame(cases_malaysia, columns= ['date','cases_new','cases_elderly','cases_adult','cases_adolescent','cases_child','cases_unvax','cases_pvax','cases_fvax'])

#get death malaysia csv and take the column we need the date is from sep 07 to mar 06 for offset 7 days
death_malaysia = pd.read_csv(death_malaysia_path)
sorted_death_malaysia=death_malaysia[death_malaysia['date'].between("2021-09-07","2022-03-06")]
sorted_death_malaysia=pd.DataFrame(sorted_death_malaysia, columns=['deaths_new'])
#categorized target into a few part
sorted_death_malaysia.index = np.arange(0, len(sorted_death_malaysia))
print(max(sorted_death_malaysia.deaths_new))

             
#get icu csv and take the column we need
icu=pd.read_csv(icu_cases_path)
sorted_icu=pd.DataFrame(icu, columns=['date','beds_icu_covid'])
sorted_icu=sorted_icu.groupby('date').sum()

#get hospital csv and take the column we need
hospital=pd.read_csv(vax_malaysia_path)
sorted_hospital=pd.DataFrame(hospital, columns=['date','beds_icu_covid'])
sorted_hospital=sorted_hospital.groupby('date').sum()

#combine all the features together for date from sep 2021 to feb 2022
dfs = [sorted_cases_malaysia, sorted_icu,sorted_hospital]
features = reduce(lambda left,right: pd.merge(left,right,on='date'), dfs)
features=features[features['date'].between("2021-09-01","2022-02-28")]



#get data from date range from sep 2021 to feb 2022
all_data= features
all_data.insert(6,"deaths_new",sorted_death_malaysia)
 
print(all_data)
print(sorted_cases_malaysia.dtypes)
print(sorted_icu.dtypes)
print(sorted_hospital.dtypes)

592
           date  cases_new  cases_elderly  cases_adult  cases_adolescent  \
189  2021-09-01      18762           1658        12434              1491   
190  2021-09-02      20988           1891        13868              1796   
191  2021-09-03      19378           1689        12793              1549   
192  2021-09-04      19057           1801        12309              1668   
193  2021-09-05      20396           1869        13561              1622   
..          ...        ...            ...          ...               ...   
365  2022-02-24      32070           2553        23538              1911   
366  2022-02-25      30644           2491        22540              1629   
367  2022-02-26      27299           2296        19914              1462   
368  2022-02-27      24466           2157        17716              1250   
369  2022-02-28      23100           2178        16748              1188   

     cases_child  deaths_new  cases_unvax  cases_pvax  cases_fvax  \
189         27